# Prompt engineering

- Create prompt w/
    - Role
    - Set categories to classify
    - Few-Shot-Examples
    - Task
    - Input for items to classify

In [87]:
import json
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

import os
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv('MISTRAL_API_KEY')


In [53]:
def run_mistral(user_message, model="mistral-medium"):
    client = MistralClient(api_key=api_key)
    messages = [
        ChatMessage(role="user", content=user_message)
    ]
    chat_response = client.chat(
        model=model,
        messages=messages
    )
    return (chat_response, chat_response.choices[0].message.content)


In [54]:
def get_rewe_categories():
    # Import product categories as dict w/ key: main category, value: list of subcategories
    with open('../data/categories_rewe.json') as f:
        categories_rewe = json.load(f)

    # Remove certain categories because they are actually labels
    exclude_categories = ['Vegane Vielfalt', 'International', 'Regional']

    for cat in exclude_categories:
        categories_rewe.pop(cat, None)

    # Include new categories needed for items that are not products
    categories_rewe['Sonstige Positionen'] = ['Pfand & Leergut', 'Rabatt & Ermäßigung', 'Kategorie nicht erkannt']

    # String categories together in a formatted string to insert in the prompt
    categories_string = list()
    for main_category in categories_rewe:
        subs_string = '## Unterkategorien\n' + '\n'.join(categories_rewe[main_category])
        categories_string.append(f'# Hauptkategorie\n{main_category}\n{subs_string}\n')
    categories_string = '\n'.join(categories_string)

    return categories_string

In [79]:
def get_prompt(item, categories):

    prompt = (
        f"""
        Du bist ein Experte für das Erkennen und Kategorisieren von verkürzten Produktnamen auf Supermarkt-Kassenbons.

        Deine Aufgabe ist die folgende:
        1. Löse den verkürzten Produktnamen in den Klammern <<< >>> zum vollständigen Produktnamen auf.
        2. Ordne das Produkt der Hauptkategorien und der dazugehörige Unterkategorie zu, die das Produkt am besten klassifiziert.

        Die möglichen Kategorien sind:

        {categories}

        Du wirst IN JEDEM FALL nur aus den vordefinierten Kategorien wählen. Deine Antwort enthält keine Erklärungen oder Anmerkungen. Die Antwort muss korrekt in JSON formatiert sein.

        ###
        Hier sind einige Beispiele:

        Verkürzter Produktname: HAUCHSCHN CURRY
        Antwort: [{{'product_name': 'Rügenwalder Mühle Veganer Hauchschnitt Typ Hähnchen', 'category_main': 'Fleisch & Fisch', 'category_sub': 'Fleischalternativen'}}]

        Verkürzter Produktname: GRANATAPEL
        Antwort: [{{'product_name': 'Granatapfel', 'category_main': 'Obst & Gemüse', 'category_sub': 'Frisches Obst'}}]

        Verkürzter Produktname: KASTEN LEER
        Antwort: [{{'product_name': 'Leergut Kasten', 'category_main': 'Sonstige Positionen', 'category_sub': 'Pfand & Leergut'}}]
        ###

        <<<
        Verkürzter Produktname: {item}
        >>>
        """
    )
    return prompt

In [80]:
categories_rewe = get_rewe_categories()
prompt = get_prompt(item='BROT VITAL GF', categories=categories_rewe)

response, message = run_mistral(prompt)

In [88]:
json.loads(response.json())

{'id': '925da901a4d3451c974b5d98aaa8f3fc',
 'object': 'chat.completion',
 'created': 1708018394,
 'model': 'mistral-medium',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': "[{'product_name': 'Brot Vital glutenfrei', 'category_main': 'Brot, Cerealien & Aufstriche', 'category_sub': 'Brot'}]"},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 1752,
  'total_tokens': 1796,
  'completion_tokens': 44}}

In [82]:
message

"[{'product_name': 'Brot Vital glutenfrei', 'category_main': 'Brot, Cerealien & Aufstriche', 'category_sub': 'Brot'}]"